In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Replace this with the path you obtained
file_path = '/Users/kaumudirawal/Desktop/crime.csv'
crime_data = pd.read_csv(file_path, encoding='ISO-8859-1')
crime_data['OCCURRED_ON_DATE'] = pd.to_datetime(crime_data['OCCURRED_ON_DATE'])

# 1) Which crime was most committed
most_common_crime = crime_data['OFFENSE_DESCRIPTION'].value_counts().idxmax()
print(f"The most committed crime is: {most_common_crime}")

# 2) Area with maximum crime
area_max_crime = crime_data['DISTRICT'].value_counts().idxmax()
print(f"The area with maximum crime is: {area_max_crime}")

# 3) Month of maximum crime
month_max_crime = crime_data['MONTH'].value_counts().idxmax()
print(f"The month with maximum crime is: {month_max_crime}")

# 4) Hour of maximum crime
hour_max_crime = crime_data['HOUR'].value_counts().idxmax()
print(f"The hour with maximum crime is: {hour_max_crime}")

# Trend over years
crime_data['YEAR'].value_counts().sort_index().plot(kind='line')
plt.title('Crime Trend Over Years')
plt.xlabel('Year')
plt.ylabel('Number of Crimes')
plt.show()

# Crime by day of the week
sns.countplot(data=crime_data, x='DAY_OF_WEEK', order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.title('Crime Count by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Crime Count')
plt.show()

# Most common crime bar plot
plt.figure(figsize=(10, 6))
crime_data['OFFENSE_DESCRIPTION'].value_counts().head(10).plot(kind='barh')
plt.title('Top 10 Most Common Crimes')
plt.xlabel('Number of Occurrences')
plt.ylabel('Crime Type')
plt.show()

# Area with maximum crime bar plot
plt.figure(figsize=(10, 6))
crime_data['DISTRICT'].value_counts().plot(kind='bar')
plt.title('Crimes by District')
plt.xlabel('District')
plt.ylabel('Number of Crimes')
plt.show()

# Month with maximum crime bar plot
plt.figure(figsize=(10, 6))
crime_data['MONTH'].value_counts().sort_index().plot(kind='bar')
plt.title('Crimes by Month')
plt.xlabel('Month')
plt.ylabel('Number of Crimes')
plt.show()

# Hour with maximum crime bar plot
plt.figure(figsize=(10, 6))
crime_data['HOUR'].value_counts().sort_index().plot(kind='bar')
plt.title('Crimes by Hour')
plt.xlabel('Hour')
plt.ylabel('Number of Crimes')
plt.show()


GeoSpatial Analysis:

In [ ]:
import folium

# Create a map centered around an average location (excluding NaN values)
boston_map = folium.Map(location=[crime_data['Lat'].mean(), crime_data['Long'].mean()], zoom_start=12)

# Filter out rows where either 'Lat' or 'Long' is NaN
filtered_crime_data = crime_data.dropna(subset=['Lat', 'Long'])

# Plot each crime incident as a point on the map
for _, row in filtered_crime_data.iterrows():
    folium.CircleMarker(location=[row['Lat'], row['Long']],
                        radius=3,
                        color='red',
                        fill=True,
                        fill_color='red',
                        fill_opacity=0.5,
                        weight=1).add_to(boston_map)

# Display the map
boston_map


Time Series Forecasting

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller 
from statsmodels.tsa.arima.model import ARIMA
from matplotlib import pyplot as plt

# Data Preparation
crime_data['OCCURRED_ON_DATE'] = pd.to_datetime(crime_data['OCCURRED_ON_DATE'])
crime_data.set_index('OCCURRED_ON_DATE', inplace=True)
monthly_crimes = crime_data.resample('M').size()

# Stationarity Check
result = adfuller(monthly_crimes)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

# If the data is not stationary, differencing might be necessary
monthly_crimes_diff = monthly_crimes.diff().dropna()

# ARIMA Model
model = ARIMA(monthly_crimes_diff, order=(1,1,1)) # these are example parameters
model_fit = model.fit()

# Forecast
plt.figure(figsize=(10,5))
plt.plot(monthly_crimes_diff, label='Original')
plt.plot(model_fit.fittedvalues, color='red', label='Fitted values')
plt.title('ARIMA Model Fit')
plt.legend()
plt.show()
